In [1]:
import pandas as pd
import datetime
from datetime import date
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import requests

In [2]:
def get_data():
    # Read Data
    pd.options.display.float_format = '{:,.3f}'.format
    f = open("../ticker.csv", "r")
    tickers = f.read().split(",")
    df = yf.download(" ".join(tickers), period = "2y")
    df_close = df["Adj Close"]
    latest_index = df_close["^GSPC"].iloc[-1]
    ratios = df_close.iloc[:].div(df_close["^GSPC"], axis = 0) * latest_index # Better way to do this?
    return ratios

def get_analysis(ratios):
    three_wk_mask = date.today()-ratios.index.date <= datetime.timedelta(days=21)
    three_m_mask = date.today()-ratios.index.date <= datetime.timedelta(days=91)
    one_yr_mask = date.today()-ratios.index.date <= datetime.timedelta(days=365)
    #st_multiplier = 2

    # Format Analysis
    analysis = pd.DataFrame(ratios.iloc[-1])
    analysis.rename({analysis.columns[0]:"latest"}, axis = 1, inplace = True) # Better way to do this?
    analysis["three_wk_high"] = pd.DataFrame(ratios[three_wk_mask].max())
    analysis["three_wk_low"] = pd.DataFrame(ratios[three_wk_mask].min())
    analysis["three_m_high"] = pd.DataFrame(ratios[three_m_mask].max())
    analysis["three_m_low"] = pd.DataFrame(ratios[three_m_mask].min())
    analysis["one_yr_high"] = pd.DataFrame(ratios[one_yr_mask].max())
    analysis["one_yr_low"] = pd.DataFrame(ratios[one_yr_mask].min())
    # Better way to do this?
    analysis["three_wk_range"] = (analysis["latest"] - analysis["three_wk_low"]) / ((analysis["three_wk_high"] - analysis["three_wk_low"]))
    analysis["three_m_range"] = (analysis["latest"] - analysis["three_m_low"]) / ((analysis["three_m_high"] - analysis["three_m_low"]))
    analysis["one_yr_range"] = (analysis["latest"] - analysis["one_yr_low"]) / ((analysis["one_yr_high"] - analysis["one_yr_low"]))
    analysis["short_term_stochastic"] = analysis["one_yr_range"] - analysis["three_wk_range"]
    analysis["long_term_stochastic"] = analysis["one_yr_range"] - analysis["three_m_range"]
    #analysis["short_term_potential"] = 1-analysis["latest"] / analysis["three_wk_high"]
    #analysis["long_term_potential"] = 1-analysis["latest"] / analysis["three_m_high"]
    st_equilibrium = (analysis["three_wk_low"] * analysis["one_yr_high"] - analysis["one_yr_low"] * analysis["three_wk_high"]) / \
        (analysis["one_yr_high"] - analysis["three_wk_high"] + analysis["three_wk_low"] - analysis["one_yr_low"])
    real_st_potential = st_equilibrium / analysis["latest"] - 1
    analysis["st_potential"] = real_st_potential
    lt_equilibrium = (analysis["three_m_low"] * analysis["one_yr_high"] - analysis["one_yr_low"] * analysis["three_m_high"]) / \
        (analysis["one_yr_high"] - analysis["three_m_high"] + analysis["three_m_low"] - analysis["one_yr_low"])
    real_lt_potential = lt_equilibrium / analysis["latest"] - 1
    analysis["lt_potential"] = real_lt_potential
    st_multiplier = abs((1 - analysis["short_term_stochastic"]) / real_st_potential).min()
    analysis["short_term_metric"] = analysis["short_term_stochastic"] + st_multiplier * real_st_potential
    lt_multiplier = abs((1 - analysis["long_term_stochastic"]) / real_lt_potential).min()
    analysis["long_term_metric"] = analysis["long_term_stochastic"] + lt_multiplier * real_lt_potential
    # Save file

    # Return output
    return analysis

def run_analysis():
    ratios = get_data()
    output = get_analysis(ratios)
    return output

In [4]:
%%time
ratios = get_data()
output = get_analysis(ratios)
output
output.sort_values(by="short_term_metric",ascending=False,inplace=True)
my_shortterm = output.loc[["DG","BLK","ZBRA","QRVO","AMD","LULU","MSFT"]]
my_longterm = output.loc[["AMGN","AAPL","WMT","OKE","INTC","KLAC","REGN"]]
output

[*********************100%***********************]  537 of 537 completed
Wall time: 37.9 s


,latest,three_wk_high,three_wk_low,three_m_high,three_m_low,one_yr_high,one_yr_low,three_wk_range,three_m_range,one_yr_range,short_term_stochastic,long_term_stochastic,st_potential,lt_potential,short_term_metric,long_term_metric
PAYC,309.000,335.298,309.000,335.298,235.173,335.298,202.627,0.000,0.737,0.802,0.802,0.064,0.085,0.085,1.000,0.210
REGN,620.860,648.973,620.860,662.067,559.263,662.067,294.073,0.000,0.599,0.888,0.888,0.289,0.044,0.066,0.989,0.402
ABBV,96.830,101.752,96.830,101.752,89.894,101.752,66.518,0.000,0.585,0.860,0.860,0.275,0.051,0.051,0.979,0.362
TMUS,107.050,113.809,106.959,113.809,96.018,113.809,75.279,0.013,0.620,0.825,0.811,0.204,0.063,0.063,0.958,0.312
BLK,554.080,566.132,554.080,571.021,500.960,571.021,435.853,0.000,0.758,0.875,0.875,0.116,0.021,0.031,0.923,0.169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CINF,70.480,70.480,64.153,96.530,54.246,123.832,54.246,1.000,0.384,0.233,-0.767,-0.151,-0.076,-0.230,-0.943,-0.545
AEP,84.150,84.274,81.817,95.854,81.817,116.208,81.817,0.949,0.166,0.068,-0.882,-0.098,-0.028,-0.028,-0.946,-0.146
AEE,74.540,74.540,71.070,87.335,71.070,100.490,71.070,1.000,0.213,0.118,-0.882,-0.095,-0.047,-0.047,-0.990,-0.175
PNW,78.260,78.260,73.480,90.882,73.480,109.471,73.480,1.000,0.275,0.133,-0.867,-0.142,-0.061,-0.061,-1.009,-0.246


In [ ]:
output.sort_values(by="long_term_metric",ascending=False,inplace=True)
output

In [ ]:
my_longterm

In [ ]:
my_shortterm

In [ ]:
ratios

In [ ]:
plt.close('all')
ratios["TSLA"].tail(250).plot()

In [ ]:
# header
uhdjejf

# header
uhdjejf